In [1]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaTokenizer, BitsAndBytesConfig, StoppingCriteriaList, TextIteratorStreamer
from transformers import pipeline


In [2]:
adapters_name  = "../experiments/jais/checkpoint-170000"
model_name = "/home/abdelrahman.sadallah/.cache/huggingface/hub/jais"

# Bits and Bytes config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # load_in_4bit=True,

    trust_remote_code=True,
    device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')

model = PeftModel.from_pretrained(model, adapters_name)
model.eval()

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): JAISLMHeadModel(
      (transformer): JAISModel(
        (wte): Embedding(84992, 5120)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-39): 40 x JAISBlock(
            (ln_1): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (attn): JAISAttention(
              (c_attn): Linear4bit(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=15360, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
      

In [3]:
orginial = '''
دَع عَنكَ لَومي فَإِنَّ اللَومَ إِغراءُ
وَداوِني بِالَّتي كانَت هِيَ الداءُ
صَفراءُ لا تَنزَلُ الأَحزانُ ساحَتَها
لَو مَسَّها حَجَرٌ مَسَّتهُ سَرّاءُ
مِن كَفِّ ذاتِ حِرٍ في زِيِّ ذي ذَكَرٍ
لَها مُحِبّانِ لوطِيٌّ وَزَنّاءُ
قامَت بِإِبريقِها وَاللَيلُ مُعتَكِرٌ
فَلاحَ مِن وَجهِها في البَيتِ لَألَاءُ
فَأَرسَلَت مِن فَمِ الإِبريقِ صافِيَةً
كَأَنَّما أَخذُها بِالعَينِ إِغفاءُ
رَقَّت عَنِ الماءِ حَتّى ما يُلائِمُها
لَطافَةً وَجَفا عَن شَكلِها الماءُ
فَلَو مَزَجتَ بِها نوراً لَمازَجَها
حَتّى تَوَلَّدُ أَنوارٌ وَأَضواءُ
دارَت عَلى فِتيَةٍ دانَ الزَمانُ لَهُم
فَما يُصيبُهُمُ إِلّا بِما شاؤوا
لِتِلكَ أَبكي وَلا أَبكي لِمَنزِلَةٍ
كانَت تَحُلُّ بِها هِندٌ وَأَسماءُ
حاشا لِدُرَّةَ أَن تُبنى الخِيامُ لَها
وَأَن تَروحَ عَلَيها الإِبلُ وَالشاءُ
فَقُل لِمَن يَدَّعي في العِلمِ فَلسَفَةً
حَفِظتَ شَيئاً وَغابَت عَنكَ أَشياءُ
لا تَحظُرِ العَفوَ إِن كُنتَ اِمرَأً حَرِجاً
فَإِنَّ حَظرَكَهُ في الدينِ إِزراءُ.
'''

In [4]:
from pyarabic.araby import strip_tashkeel
original = strip_tashkeel(orginial)
print(original)


دع عنك لومي فإن اللوم إغراء
وداوني بالتي كانت هي الداء
صفراء لا تنزل الأحزان ساحتها
لو مسها حجر مسته سراء
من كف ذات حر في زي ذي ذكر
لها محبان لوطي وزناء
قامت بإبريقها والليل معتكر
فلاح من وجهها في البيت لألاء
فأرسلت من فم الإبريق صافية
كأنما أخذها بالعين إغفاء
رقت عن الماء حتى ما يلائمها
لطافة وجفا عن شكلها الماء
فلو مزجت بها نورا لمازجها
حتى تولد أنوار وأضواء
دارت على فتية دان الزمان لهم
فما يصيبهم إلا بما شاؤوا
لتلك أبكي ولا أبكي لمنزلة
كانت تحل بها هند وأسماء
حاشا لدرة أن تبنى الخيام لها
وأن تروح عليها الإبل والشاء
فقل لمن يدعي في العلم فلسفة
حفظت شيئا وغابت عنك أشياء
لا تحظر العفو إن كنت امرأ حرجا
فإن حظركه في الدين إزراء.



In [15]:
def gen(model, text: str, tmp=.8, top_p=0.8, max_length=256):
    inputs = tokenizer(text, return_tensors="pt").to('cuda')
    inputs_length = len(inputs["input_ids"][0])
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=max_length,
                                do_sample=True,
                                top_p=top_p,
                                temperature=tmp,
                                repetition_penalty=1.3
                                )

    return tokenizer.decode(outputs[0][inputs_length:], skip_special_tokens=True)

In [16]:
base_text = '''### Instruction: Generate a poem based on the following title, and the given era:

### Input:
'''

text = '''دع عنك لومي فان اللوم اغراء 
و داوني بالتي كانت هي الداء'''

# text = ''' أصبح الملك للذي فطر الخلق , العصر العثماني'''

# base_text = ''''''

# Happy kid was playing at the park, but then he broke his leg, and his life got completely changed.

with open('../results/arabic_output.txt', 'a') as f:
    while 1:
        print('-'*20)
        # text = input("Enter a prompt: ")
        print('-'*20)

        text = base_text + text

        # text = 'who are you?'
        
        f.write('='*30 + '\n')
        f.write(text + '\n')
        f.write('-'*20 + '\n')


    
        output = gen(model, text)

        output = output.replace('<newline>', '\n')

        f.write(output + '\n')
        f.write('='*30 + '\n')
        f.flush()

        print('-'*20)
        print(output)
        
        print('-'*20)


--------------------
--------------------
--------------------
ا, العصر العباسي

### Response:
دعني فداؤك ما بالغيض من طمع
من كان يحسب ان الغيض غضاءا
ما كنت احسب أن الغدر عادته
حتى رميت بنبل منك حاءى
يا رب يوم قد اعرتك مطامعي
وما استعرت ولم تستفد رجاءا
كنت امرءا لو غدوت البحر لم يكن
مهما سواك ولا شفى داءه الشفا
فكيف صرت الى خسف وانحطاط ومن
خفت عليه ذا جاءك الخافوا
لولا هواي لأصبحت في خلقي كما
كانت قريش على قوم لهم شرفا
أمسكت عن مدحهم حتى كفوك وما
في الكف بعد المنع لا الشكر والجزاءا
لو عاينتني غداة البين يا أملي
والعين تنظرني ذ ذاك ملقى ومثنى
لا شك أنك تعرفين الذي صنعت بي
يوم الفراق وفي التوديع كيف بكا
ولو علمت بأني لست أعطفكم
علي يوما لبادلت الوجد جفاءا
لكن رأيت زماني فيك مغتربا
قد ضل مني سبيله وتاها
وأيقنت أني ون طال المدى سنتي
لن أحظى بطيف منكم فألقاه
الله يعلم اني كلما ذكروا ال
--------------------
--------------------
--------------------
--------------------
, العصر العباسي

### Response:
دعني فما لك من لوم على أحد
ن كان منك له نفع و صلاحا
لا تكذبن فن المرء ليس به
من صحة العقل أو 

KeyboardInterrupt: 

In [ ]:
from utils import *

train_dataset, val_dataset, test_dataset = get_arabic_datasets(field = 'prompt')

In [ ]:
print(train_dataset[0]['prompt'])

### Instruction: Generate a poem based on the following title, and the given era:

    ### Input:
    أصبح الملك للذي فطر الخلق , العصر العثماني

    ### Response:
    أصبح الملك للذي فطر الخل
ق بتقدير للعزيز العليم
غافر الذنب للمسيء بعفو
قابل التوب ذي العطاء العميم
مرسل المصطفى البشير لينا
رحمة منه بالكلام القديم
ربنا ربنا ليك أنينا
فأجرنا من حر نار الجحيم
واكفنا شر ما نخاف بلطف
يا عظيما يرجى لكل عظيم
وتقبل أعمالنا واعف عنا
وأنلنا دخول دار النعيم
بنبي بعثته فهدانا
لصراط من الهدى مستقيم
وبمن نحن في حماه مدى الدهر
أخيه يحيى الحصور الكريم
أدرك أدرك قوما أتوا بافتقار
وانكسار ومدمع مسجوم
شهدت أرواحهم أنك الله
وجاءوا بكل قلب سليم


In [ ]:


# x = train_dataset[0]['prompt']
x = "Can you tell me who are you"

inputs = tokenizer(x, return_tensors="pt").to('cuda')

out = model(**inputs, return_dict=True)

RuntimeError: CUDA error: CUBLAS_STATUS_INTERNAL_ERROR when calling `cublasGemmEx( handle, opa, opb, m, n, k, &falpha, a, CUDA_R_16F, lda, b, CUDA_R_16F, ldb, &fbeta, c, CUDA_R_16F, ldc, CUDA_R_32F, CUBLAS_GEMM_DFALT_TENSOR_OP)`

In [ ]:
labels =  torch.argmax(out.logits, dim=-1)

l = tokenizer.decode(labels[0])

print(l)

print(tokenizer.decode(inputs['input_ids'][0]))

't tell me what you you?"

Can you tell me who are you
